In [25]:
from __future__ import print_function

from pyspark import SparkContext
# $example on$
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.regression import LabeledPoint
from numpy import array
sc.stop()
if __name__ == "__main__":

    sc = SparkContext(appName="p3dt")
    sc.setLogLevel("ERROR")
    m1 = DecisionTreeModel.load(sc, "runs")
    m2 = DecisionTreeModel.load(sc, "wickets")
    batsmen_cluster={}
    bowler_cluster={}
    with open('cluster_batsmen.csv') as batclus:
        for line in batclus:
            arr=line.split(',')
            aa=[]
            aa.append(int(arr[0]))
            aa.append(float(arr[3]))
            aa.append(float(arr[4]))
            batsmen_cluster[arr[2]]=aa
    with open('cluster_bowler.csv') as bowlclus:
        for line in bowlclus:
            arr=line.split(',')
            aa=[]
            aa.append(int(arr[0]))
            aa.append(int(arr[3]))
            aa.append(float(arr[4]))
            bowler_cluster[arr[2]]=aa
    listoffiles=['3.csv']
    for files in listoffiles:
        with open('input/'+files,'r') as fi:
            a=0
            for l in fi:
                a=a+1
                if(a==2):
                    TEAM1=l.split(',')[0]
                if(a==3):
                    TEAM2=l.split(',')[0]
                if(a==6):
                    SCORE1=int(l.split(',')[1])
                if(a==11):
                    SCORE2=int(l.split(',')[1])
        TEAM1=TEAM1.ljust(30)
        TEAM2=TEAM2.ljust(30)
        print(TEAM1,' REAL SCORE:',SCORE1)
        print(TEAM2,' REAL SCORE:',SCORE2)
        if(SCORE1>SCORE2):
            winner=TEAM1
        elif(SCORE1<SCORE2):
            winner=TEAM2
        else:
            winner="BOTH"
        print(winner," won")
        team1score=0
        for z in range(1,3):
            with open('input/'+files,'r') as fi:
                i=0
                for line in fi:
                    i=i+1
                    if((i==4 and z==1) or (i==9 and z==2)):
                        bat=line.split(',')
                        bat=[x.strip('\n') for x in bat]
                    if((i==5 and z==1) or (i==10 and z==2)):
                        bowl=line.split(',')
                        bowl=[x.strip('\n') for x in bowl]	
            A=bat.pop(0)
            B=bat.pop(0)
            score=0
            for i in range(1,len(bowl)+1):
                X=bowl.pop(0)
                bat1=batsmen_cluster[A]
                bat2=batsmen_cluster[B]
                bow=bowler_cluster[X]
                test=sc.parallelize([array(bat1+bat2+bow)])
                predictions=m1.predict(test)
                results=predictions.collect()
                runs=int(round(results[0]))
                predictions=m2.predict(test)
                results=predictions.collect()
                wickets=int(results[0])
                score=score+runs
                if(z==2 and score>team1score):
                    break
                for i in range(wickets):
                    if not A:
                        break
                    A=B
                    B=bat.pop(0)
                if(runs%2!=0):
                    t=A
                    A=B
                    B=t
            if(z==1):
                team1score=score
                print(TEAM1,' PREDICTED SCORE',score)
            else:
                if(team1score>score):
                    w=TEAM1
                elif(team1score<score):
                    w=TEAM2
                else:
                        w="BOTH"
                print(TEAM2,' PREDICTED SCORE',score,' : ',w)
                print('\n\n')
    sc.stop()


Mumbai Indians                  REAL SCORE: 213
Royal Challengers Bangalore     REAL SCORE: 167
Mumbai Indians                  won
Mumbai Indians                  PREDICTED SCORE 520
Royal Challengers Bangalore     PREDICTED SCORE 380  :  Mumbai Indians                



